# Deep Deterministic Policy Gradients 

In [98]:
import  gym
import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
import numpy as np  

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from IPython.display import clear_output

In [99]:
env = gym.make('HalfCheetah-v4', render_mode = "human")
env.reset()
env.render()

In [100]:
env.action_space

Box(-1.0, 1.0, (6,), float32)

In [101]:
DIM_ACTIONS = 6

### Exploring the observation space 

In [102]:
observation_space = env.observation_space
observation_space

Box(-inf, inf, (17,), float64)

In [103]:
DIM_OBSERVATIONS = 17

### Building a class for our replay buffer

In [104]:
#named tuple to store experience 
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))

#Experience replay buffer object 
class ReplayMemory(object):

    def __init__(self, capacity):
        #contains a deque of specified buffer length 
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        #add new transition to buffer 
        self.memory.append(Transition(*args))

    #simple function to sample some experience from memory randomly 
    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

### Defining our Critic (Q-Value function)

In [105]:
class critic_Q(nn.Module):

    def __init__(self, len_observation_space, dim_actions):
        super(critic_Q, self).__init__()
        #observations to 128 
        self.layer1 = nn.Linear(len_observation_space + dim_actions, 128)
        #linear transformation 
        self.layer2 = nn.Linear(128, 128)
        #linear downsizing to number of possible actions 
        self.layer3 = nn.Linear(128, 1)

    def forward(self, observation, action):
        x = torch.cat([observation, action], dim=-1)
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return self.layer3(x)

### Policy function

In [106]:
class actor_Policy(nn.Module): 
    def __init__(self, len_observation_space, dim_actions):
        super(actor_Policy, self).__init__()
        #observations to 128 
        self.layer1 = nn.Linear(len_observation_space, 128)
        #linear transformation 
        self.layer2 = nn.Linear(128, 128)
        #deterministically outputting a single action 
        self.layer3 = nn.Linear(128, dim_actions)

    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return torch.tanh(self.layer3(x))

### Optimization Function Overview

The following function represents the optimization steps. If our replay buffer is at capacity, we:

- Sample a batch of transitions from memory. This initial step involves drawing a selection of varied transitions from our repository of stored experiences.

- Compute the expected value using the policy network. This step requires us to determine probable outcomes and their associated probabilities, based on our current policy.

- Compute target values by adding the observed reward to the expected next state value. This expected next state value is calculated using the target network instead of the policy network. 

- Compute the loss between the predicted and target values and then optimize the policy network. This final step is crucial for improving the predictive accuracy of our model. It works by learning from the discrepancies between predicted and actual outcomes, and uses these insights to refine the model.

Through this sequence of steps, the function continuously enhances the model's predictive performance by learning from past errors.


In [107]:
def optimize_model(gamma, batch_size, memory, actor, critic, target_actor, target_critic, loss_fn, actor_optimizer, critic_optimizer, device):
    
    if len(memory) < batch_size:
        return
    
    transitions = memory.sample(batch_size)
    batch = Transition(*zip(*transitions)) #batches our data as follows: ((state1, state2, ..), (action1, action2, ...), ...)
    
    non_final_mask = torch.tensor([s is not None for s in batch.next_state],
                                  device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state if s is not None])
    
    states = torch.cat(batch.state)
    actions = torch.cat(batch.action)
    rewards = torch.cat(batch.reward)
    
    actions = actions.unsqueeze(-1)
    predicted_values = critic(states, actions) 
    
    with torch.no_grad():
        next_actions = target_actor(non_final_next_states)  
    
    next_state_values = torch.zeros(batch_size, device=device)
    with torch.no_grad():
        next_state_values[non_final_mask] = target_critic(non_final_next_states, next_actions)  
    
    target_values = (next_state_values * gamma) +rewards # if the state is non terminal, we add to it the expected value of the next state 
    td_error = target_values - predicted_values 
    
    loss = loss_fn(predicted_values, target_values.unsqueeze(1))
    critic_optimizer.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_value_(critic.parameters(), 100)
    critic_optimizer.step()

    log_action = torch.log(actions)
    actor_loss = - log_action * td_error
    actor_optimizer.zero_grad()
    actor_loss.backward()
    actor_optimizer.step()

In [108]:
def trainer(num_episodes, batch_size, target_actor, actor, target_critic, critic, gamma, epsilon, device, replay_buffer, actor_optimizer, critic_optimizer, tau): 
    
    reward_per_episode = []

    for episode in range(num_episodes): 
        
        state, _ = env.reset()
        state = torch.tensor(state, dtype=torch.float32, device = device).unsqueeze(0)
        episode_end = False
        step = 0 
        cum_reward = 0

        if  not episode % 100 and episode < 400: 
            epsilon-=0.1

        while not episode_end:
            action = actor(state).squeeze().cpu().detach().numpy()
            next_state, reward, terminated, truncated, _ = env.step(action)
            
            step+=1
            cum_reward+=reward
            
            episode_end = terminated or truncated 
            
            if not terminated: 
                next_state = torch.tensor(next_state, dtype=torch.float32, \
                    device = device).unsqueeze(0) #we only care about next_state if the state is non terminal
            else: 
                next_state = None
            
            action = torch.tensor(action, dtype=torch.int64, device=device).unsqueeze(0)
            reward = torch.tensor(reward, dtype=torch.float32, device=device).unsqueeze(0)

            replay_buffer.push(state, action, next_state, reward)
            state = next_state
            
            optimize_model(gamma, batch_size, replay_buffer, actor, critic, target_actor, target_critic,loss_fn=torch.nn.MSELoss(size_average=True), actor_optimizer=actor_optimizer,\
                           critic_optimizer= critic_optimizer, device=device)
            
            if not episode % (num_episodes / 50) and episode > 50:
                #extract network parameters for both networks 
                target_critic_state_dict = target_critic.state_dict()
                policy_network_state_dict = critic.state_dict()

                target_actor_state_dict = target_actor.state_dict()
                policy_network_state_dict = actor.state_dict()
                
                for key in policy_network_state_dict.keys():
                    # θ′ ← τ θ + (1 −τ )θ′
                    target_critic_state_dict[key] = policy_network_state_dict[key]*tau + target_critic_state_dict[key]*(1-tau)
                    target_critic.load_state_dict(target_critic_state_dict)

                    target_actor_state_dict[key] = policy_network_state_dict[key]*tau + target_actor_state_dict[key]*(1-tau)
                    target_actor.load_state_dict(target_actor_state_dict)
            
        print("Episode: {} Score: {}".format(episode, cum_reward))
        reward_per_episode.append(cum_reward)
        clear_output(wait=True) # Clear the output of the current cell receiving output
        plt.plot(reward_per_episode)
        plt.title('Cumulative Reward per Episode')
        plt.xlabel('Episode')
        plt.ylabel('Cumulative Reward')
        plt.show()

In [109]:
#hyperparams 
NUM_EPISODES = 500 # this is abitrary 
E = 0.5 #we start off with a relatively high exploration rate and then decay that over training steps to start exploiting more 
GAMMA = 1 #
ALPHA = 3e-4
BATCH_SIZE = 32 #we want a relatively large batch size to leverage GPU parallelism but at the same time we want this number to be far 
                #smaller than our memory capacity so that we can sample randomly at the same time 
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [110]:
replay_buffer = ReplayMemory(100000)

actor = actor_Policy(len_observation_space=DIM_OBSERVATIONS, dim_actions=DIM_ACTIONS)
target_actor = actor_Policy(len_observation_space=DIM_OBSERVATIONS, dim_actions=DIM_ACTIONS)

critic = critic_Q(len_observation_space=DIM_OBSERVATIONS, dim_actions=DIM_ACTIONS)
target_critic = critic_Q(len_observation_space=DIM_OBSERVATIONS, dim_actions=DIM_ACTIONS)

actor.to(DEVICE)
target_actor.to(DEVICE)

critic.to(DEVICE)
target_critic.to(DEVICE)

target_actor.load_state_dict(actor.state_dict())
target_critic.load_state_dict(critic.state_dict())

<All keys matched successfully>

In [111]:
actor_optimizer = torch.optim.Adam(actor.parameters(), lr=ALPHA)

In [112]:
critic_optimizer = torch.optim.Adam(critic.parameters(), lr=ALPHA)

In [113]:
trainer(num_episodes=NUM_EPISODES, batch_size=BATCH_SIZE, actor=actor, critic=critic, target_actor=target_actor, target_critic=target_critic,\
         gamma=GAMMA, epsilon=E, device=DEVICE, replay_buffer=replay_buffer, actor_optimizer=actor_optimizer, critic_optimizer=critic_optimizer, tau=0.005)

RuntimeError: Tensors must have same number of dimensions: got 2 and 3

: 